In [6]:
import numpy as np
import sys
sys.path.insert(0, '/Users/trimcao/Dropbox/Richardson/Fall-2017/cs6375-ml-ruozzi/solution/lib')
sys.path.insert(0, '/home/trimcao/Dropbox/Richardson/Fall-2017/cs6375-ml-ruozzi/solution/lib')
import importlib as imp
# using reload to avoid having to restart the Python kernel after modifying source code
import NaiveBayes; imp.reload(NaiveBayes)
from NaiveBayes import GaussianNB

In [7]:
def read_in(file_path):
    X = []
    y = []
    with open(file_path, 'r') as f:
        for line in f:
            info = line.strip('\n').split(',')
            # label is the first column
            X.append([float(i) for i in info[:-1]])
            y.append(float(info[-1]))
    X = np.array(X)
    y = np.array(y)
    # change the output to *-1 and 1* instead of *0 and 1*
    y[y == 0] = -1
    return X, y

In [8]:
X_train, y_train = read_in('hw4_data/spam_train.data')
X_test, y_test = read_in('hw4_data/spam_test.data')
X_valid, y_valid = read_in('hw4_data/spam_validation.data')

In [9]:
def normalize_data(X, X_train):
    X_norm = X - X_train.mean(axis=0)
    X_norm = X_norm / X_train.std(axis=0)
    return X_norm

### Gaussian Naive Bayes

In [25]:
clf = GaussianNB()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print(np.mean(preds==y_test))
print(1-np.mean(preds==y_test))

0.595505617978
0.404494382022


### PCA as feature selection

In [11]:
# construct the W matrix with zero mean
W = normalize_data(X_train, X_train)
W_test = normalize_data(X_test, X_train)

In [12]:
k = 10
s = 10
D = X_train.shape[1]

In [13]:
cov_mat = np.cov(W.T)

In [14]:
eig_vals, eig_vecs = np.linalg.eigh(cov_mat)
idx = eig_vals.argsort()[::-1]   
eig_vals = eig_vals[idx]
eig_vecs = eig_vecs[:,idx]

In [15]:
print(eig_vals[:k])

[ 6.67647395  3.26013288  2.25116837  1.84721798  1.63959314  1.59257407
  1.51572868  1.41980701  1.40200761  1.30402369]


In [16]:
top_vecs = eig_vecs[:k]

In [17]:
print(top_vecs.shape)

(10, 57)


In [18]:
# feature distribution, aka Pi
feature_dist = np.sum(top_vecs**2, axis=0) / k

In [19]:
np.sum(feature_dist)

1.0000000000000004

In [20]:
features = np.random.choice(D, s, p=feature_dist)
features

array([13, 32, 36, 35, 19, 46, 10, 40, 13,  7])

In [21]:
features = np.unique(features)
features

array([ 7, 10, 13, 19, 32, 35, 36, 40, 46])

In [22]:
def feature_select(k, s, eig_vals, eig_vecs):
    """
    Select s features using the top k eigenvectors.
    eig_vals and eig_vecs must be sorted.
    Return an array of features indices.
    """
    idx = eig_vals.argsort()[::-1]   
    eig_vals = eig_vals[idx]
    eig_vecs = eig_vecs[:,idx]
    top_vecs = eig_vecs[:k]
    feature_dist = np.sum(top_vecs**2, axis=0) / k
    # sample s features using np.random.choice
    features = np.random.choice(D, s, p=feature_dist)
    # get the unique feature indices
    features = np.unique(features)
    return features

In [24]:
for k in range(1,11):
    for s in range(1,21):
        error = 0
        for _ in range(100):
            features = feature_select(k,s,eig_vals,eig_vecs)
            clf = GaussianNB()
            clf.fit(W[:,features], y_train)
            preds = clf.predict(W_test[:,features])
            error += 1 - np.mean(preds==y_test)
        ave_error = error/100
        print('k = ' + str(k) + ', s = ' + str(s) + ': average error is ' +
                 str(ave_error))

k = 1, s = 1: average error is 0.656529338327
k = 1, s = 2: average error is 0.733620474407
k = 1, s = 3: average error is 0.745605493134
k = 1, s = 4: average error is 0.775855181024
k = 1, s = 5: average error is 0.784719101124
k = 1, s = 6: average error is 0.77606741573
k = 1, s = 7: average error is 0.798888888889
k = 1, s = 8: average error is 0.741061173533
k = 1, s = 9: average error is 0.773870162297
k = 1, s = 10: average error is 0.785193508115
k = 1, s = 11: average error is 0.780074906367
k = 1, s = 12: average error is 0.774806491885
k = 1, s = 13: average error is 0.755855181024
k = 1, s = 14: average error is 0.752172284644
k = 1, s = 15: average error is 0.74759051186
k = 1, s = 16: average error is 0.726679151061
k = 1, s = 17: average error is 0.732297128589
k = 1, s = 18: average error is 0.743732833958
k = 1, s = 19: average error is 0.718963795256
k = 1, s = 20: average error is 0.730998751561
k = 2, s = 1: average error is 0.593995006242
k = 2, s = 2: average err

k = 9, s = 18: average error is 0.713632958801
k = 9, s = 19: average error is 0.718202247191
k = 9, s = 20: average error is 0.673133583021
k = 10, s = 1: average error is 0.585468164794
k = 10, s = 2: average error is 0.695555555556
k = 10, s = 3: average error is 0.766254681648
k = 10, s = 4: average error is 0.687852684145
k = 10, s = 5: average error is 0.717141073658
k = 10, s = 6: average error is 0.666279650437
k = 10, s = 7: average error is 0.669413233458
k = 10, s = 8: average error is 0.726716604245
k = 10, s = 9: average error is 0.677041198502
k = 10, s = 10: average error is 0.706516853933
k = 10, s = 11: average error is 0.743283395755
k = 10, s = 12: average error is 0.727540574282
k = 10, s = 13: average error is 0.697640449438
k = 10, s = 14: average error is 0.694731585518
k = 10, s = 15: average error is 0.697353308365
k = 10, s = 16: average error is 0.720362047441
k = 10, s = 17: average error is 0.698764044944
k = 10, s = 18: average error is 0.707016229713
k = 

In [ ]:
features = feature_select(k,s,eig_vals,eig_vecs)
X_choice = X_train[:,features]
clf = GaussianNB()
clf.fit(X_choice, y_train)
preds = clf.predict(X_test[:,features])
print(np.mean(preds==y_test))

Best k/s pair: k = 5, s = 1: average error is 0.440599250936

Disadvantages:
- Hard to choose k and s beforehand, and it takes a lot of time to do grid search